In [2]:
import dask
dask.config.set({'dataframe.query-planning': False})

import h5py
from pathlib import Path
from functools import reduce
import pandas as pd
import sys
sys.path.append("../../../workflow/scripts/")
import readwrite
cfg = readwrite.config()

In [39]:
# get chromium refs gene panels
seurat_to_h5_dir = Path(cfg['results_dir']) / 'seurat_to_h5'
gene_panels = {}
for dir in seurat_to_h5_dir.iterdir():
    if dir.is_dir():
        if 'lung' in dir.name and 'external' not in dir.name:
            with h5py.File(dir / 'RNA_counts.h5', 'r') as f:
                gene_panels[dir.name] = f[list(f.keys())[0]]['gene_names'].asstr()[:].tolist()


# get xenium gene panels info
u_condition_panel = [
    ['NSCLC','5k'],
    ['NSCLC','lung']
 ]
for condition_panel in u_condition_panel:
    condition, panel = condition_panel
    name = '/'.join(condition_panel)
    panel_path = Path(cfg['xenium_processed_data_dir'] + f'10x_5um/{condition}/{panel}')
    
    # get gene panel info from first sample
    donor = list(panel_path.iterdir())[0]
    sample = list(donor.iterdir())[0]
    p = sample / 'normalised_results/outs/gene_panel.json'

    df = readwrite.get_gene_panel_info(p)
    gene_panels[condition, panel] = df[df['id'].notna()]['name'].tolist()


# get common genes between lung, 5k, chromium
common_genes = list(reduce(set.intersection, map(set, gene_panels.values())))
common_genes = pd.DataFrame({'gene': common_genes})
# common_genes.to_csv(cfg['markers_dir']+'Xenium_NSCLC_5k_lung_chromium_common_genes.csv')